In [1]:
import torch
from torch import nn
import torch.utils.data as Data
import torch.optim as optim
from torch.nn import init
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
from IPython import display
from matplotlib import pyplot as plt


In [2]:
train_dataset = torchvision.datasets.MNIST(root='./Datasets/MNIST', train=True,  download=True, transform=transforms.ToTensor())  
test_dataset = torchvision.datasets.MNIST(root='./Datasets/MNIST', train=False,  download=True, transform=transforms.ToTensor())  


1.7%

100.0%


Extracting ./Datasets/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./Datasets/MNIST/MNIST/raw


100.0%

Extracting ./Datasets/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./Datasets/MNIST/MNIST/raw



100.0%


Extracting ./Datasets/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./Datasets/MNIST/MNIST/raw



100.0%

Extracting ./Datasets/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./Datasets/MNIST/MNIST/raw



In [3]:
batch_size = 32  
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,  num_workers=0)  
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False,  num_workers=0) 


In [4]:

input_num=784 #28*28
hidden_num=256
output_num=10

class FlattenLayer(nn.Module):
    def _init_(self):
        super(FlattenLayer,self)._init_()
    def forward(self, x):
        return x.view(x.shape[0],-1)
#模型定义
net1 = nn.Sequential(
        FlattenLayer(),
        nn.Linear(input_num,hidden_num),
        nn.ReLU(),
        # nn.ELU()
        # nn.Sigmoid()
        nn.Linear(hidden_num,output_num)
    )

net2 = nn.Sequential(
        FlattenLayer(),
        nn.Linear(input_num,hidden_num),
        # nn.ReLU(),
        nn.ELU(),
        # nn.Sigmoid()
        nn.Linear(hidden_num,output_num)
    )

net3 = nn.Sequential(
        FlattenLayer(),
        nn.Linear(input_num,hidden_num),
        # nn.ReLU(),
        # nn.ELU()
        nn.Sigmoid(),
        nn.Linear(hidden_num,output_num)
    )


In [5]:
for param in net1.parameters():
    init.normal_(param,mean=0,std=0.01)

for param in net2.parameters():
    init.normal_(param,mean=0,std=0.01)

for param in net3.parameters():
    init.normal_(param,mean=0,std=0.01)


In [6]:
lr=0.01
loss = torch.nn.CrossEntropyLoss()
optimizer1 = optim.SGD(net1.parameters(),lr)
optimizer2 = optim.SGD(net2.parameters(),lr)
optimizer3 = optim.SGD(net3.parameters(),lr)


In [7]:
def evaluate_accuracy_loss(net, data_iter):
    acc_sum=0.0
    loss_sum=0.0
    n=0
    for X,y in data_iter:
        y_hat = net(X)
        acc_sum += (y_hat.argmax(dim=1)==y).sum().item()
        l = loss(y_hat,y)
        loss_sum += l.item()*y.shape[0]
        n+=y.shape[0]
    return acc_sum/n,loss_sum/n


In [8]:

def train3 (net,train_iter,test_iter,loss,epoch_num,batch_size,params = None,lr=None,optimizer=None):
    train_loss=[]
    test_loss=[]
    for epoch in range(epoch_num):
        train_l_sum=0.0
        train_acc_num=0.0
        n =0.0
        for X,y in train_iter:
            y_hat = net(X)
            
            l = loss(y_hat,y)
            #梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            if optimizer is None:
                SGD(params,lr,batch_size)
            else:
                optimizer.step()
            
            train_l_sum += l.item()*y.shape[0]
            
            train_acc_num += (y_hat.argmax(dim=1)==y).sum().item()
            n += y.shape[0]
     
        train_loss.append(train_l_sum/n)#训练集loss
        test_acc,test_l = evaluate_accuracy_loss(net,test_iter)
        test_loss.append(test_l)
        print("epoch %d,train_loss %.6f,test_loss %.6f,train_acc %.6f,test_acc %.6f"%(epoch+1,train_loss[epoch],test_loss[epoch],train_acc_num/n,test_acc)) 
    return train_loss, test_loss


In [ ]:
lr = 0.01
epoch_num=100
train_loss,test_loss=train3(net1,train_iter,test_iter,loss,epoch_num,batch_size,net1.parameters(),lr,optimizer1)